In [19]:

using Revise
includet("../general_modules/channels.jl")
include("../general_modules/mesh.jl")
using .channels
using .mesh
using JSON

In [ ]:
data = JSON.parsefile("test.json")
Systems_params= data["Systems"]
mesh_params= data["Mesh"]
fermion=false

α= α3b(fermion,0.0,0.0,1,Systems_params["lmax"],Systems_params["lmin"],Systems_params["λmax"],Systems_params["λmin"],Systems_params["s1"],Systems_params["s2"],Systems_params["s3"],Systems_params["t1"],Systems_params["t2"],Systems_params["t3"],Systems_params["MT"])

grid= initialmesh(mesh_params["nθ"],mesh_params["nx"],mesh_params["ny"],Float64(mesh_params["xmax"]),Float64(mesh_params["ymax"]),Float64(mesh_params["alpha"]))

For J=0.0 T=0.0 parity=1 Number of channels: 0
scaling factor for x: 0.15032074721240762


Main.mesh.meshset(20, 20, 20, [-0.9931285991850949, -0.9639719272779138, -0.912234428251326, -0.8391169718222188, -0.7463319064601508, -0.636053680726515, -0.5108670019508271, -0.37370608871541955, -0.2277858511416451, -0.07652652113349732, 0.07652652113349732, 0.2277858511416451, 0.37370608871541955, 0.5108670019508271, 0.636053680726515, 0.7463319064601508, 0.8391169718222188, 0.912234428251326, 0.9639719272779138, 0.9931285991850949], [0.017614007139152055, 0.04060142980038697, 0.06267204833410904, 0.08327674157670457, 0.10193011981724034, 0.11819453196151841, 0.13168863844917655, 0.142096109318382, 0.14917298647260377, 0.15275338713072587, 0.15275338713072587, 0.14917298647260377, 0.142096109318382, 0.13168863844917655, 0.11819453196151841, 0.10193011981724034, 0.08327674157670457, 0.06267204833410904, 0.04060142980038697, 0.017614007139152055], [0.010603608926780523, 0.055938381339777916, 0.13778130652680518, 0.25664359346480425, 0.41326168629390836, 0.6086374785852973, 0.84407729

In [ ]:
for i in 1:mesh_params["nx"]
    println(grid.xi[i], "   ", grid.dxi[i])
end

In [15]:
using Revise
Revise.revise()
includet("Gcoefficient.jl")
using .Gcoefficient

initialY(λmax, lmax, nθ, nx, ny, cosθi, xi, yi, P::Char)
return Yλαout, Ylαin, Yλαin

Yλin[iθ, iy, ix, nch, perm_index]

In [17]:
Gαα=computeGcoefficient(α, grid)

20×20×20×0×0×2 Array{ComplexF64, 6}

In [10]:
using Revise
includet("twobody.jl")
using .twobodybound

In [14]:
potname="MT"
bound_states=bound2b(grid, potname)

For J=1.0 parity=+ Number of channels: 2
Number of bound states: 1
Bound state energies: [-2.183807213848866]


1-element Vector{Any}:
 (-2.183807213848866, [9.374102359288964e-5, 0.0012976542070245632, 0.006317508976450265, 0.022264673318424077, 0.057719247149088174, 0.11726384798501614, 0.19089002101713984, 0.2617542672238449, 0.31321869382684275, 0.3417805336714798  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [ ]:
using Revise
includet("spline.jl")
using .spline

In [ ]:
"""
Test the cubic Hermitian spline interpolation with a polynomial function.
"""
# Import Printf for formatting
using Printf
using Plots
function test_cubherm_interpolation()
    # Define the polynomial function f(x) = 5x^5 + 4x^4 + 3x^3 + 2x^2 + x
    f(x) = 5*x^5 + 4*x^4 + 3*x^3 + 2*x^2 + x
    
    # Create a set of old grid points (coarse grid)
    xold = collect(range(-1.0, 1.0, length=15))
    
    # Evaluate the function at the old grid points
    yold = f.(xold)
    
    # println("Original grid points:")
    # for i in 1:length(xold)
    #     println("x = $(xold[i]), f(x) = $(yold[i])")
    # end
    # println()
    
    # Create a set of test points (fine grid) where we want to interpolate
    xtest = collect(range(-0.95, 0.95, length=20))
    
    # Compute the exact values
    yexact = f.(xtest)
    
    # Interpolate using our cubic Hermitian spline
    yinterp = Float64[]
    for x in xtest
        push!(yinterp, cubherm_interp_point(xold, yold, x))
    end
    
    # Compute errors
    errors = abs.(yinterp - yexact)
    max_error = maximum(errors)
    avg_error = sum(errors) / length(errors)
    
    println("Interpolation results and errors:")
    println("   x       Exact        Interpolated       Error")
    println("------------------------------------------------------")
    for i in 1:length(xtest)
        # Use simple string formatting instead of @printf/@sprintf
        x_str = @sprintf("%.4f", xtest[i])
        exact_str = @sprintf("%.6f", yexact[i])
        interp_str = @sprintf("%.6f", yinterp[i])
        error_str = @sprintf("%.6e", errors[i])
        
        println("$(lpad(x_str, 7))  $(lpad(exact_str, 12))  $(lpad(interp_str, 12))  $(lpad(error_str, 12))")
    end
    
    println("\nMaximum error: $max_error")
    println("Average error: $avg_error")
    
    # Optional: Plot the results if Plots.jl is available
    try
        
        
        p = plot(xold, yold, seriestype=:scatter, 
             label="Original Points", markersize=6, legend=:topleft)
        
        plot!(p, xtest, yexact, 
              label="Exact Function", linewidth=2)
        
        plot!(p, xtest, yinterp, 
              label="Interpolated", linewidth=2, linestyle=:dash)
        
        # Plot error
        p_error = plot(xtest, errors, 
                   label="Interpolation Error", linewidth=2,
                   xlabel="x", ylabel="Error", title="Interpolation Error")
        
        # Display both plots
        plot(p, p_error, layout=(2,1), size=(800, 600),
             title=["Cubic Hermitian Spline Interpolation" ""])
        
        savefig("interpolation_test.png")
        println("\nPlot saved as 'interpolation_test.png'")
    catch
        println("\nPlotting skipped (Plots.jl not available)")
    end
    
    return max_error, avg_error
end

In [ ]:
using Printf 
test_cubherm_interpolation()

In [ ]:
using Revise
includet("matrices.jl")
using .matrices


In [ ]:
Rxy_matrix(α, grid)


In [ ]:
using Revise
includet("threebodybound.jl")
using .threebodybound

In [ ]:
ThreeBody_Bound(α, grid)

In [ ]:
T_matrix(α,grid) 

In [ ]:
V_matrix(α, grid, "AV18")